In [1]:
from sklearn.linear_model import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import numpy as np
import pandas as pd


import tensorflow as tf

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [21]:

from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [2]:
data_file = pd.read_csv("../TAMU_FINAL_DATASET_2018.csv")

In [4]:
# only select the numeric data
df = data_file.select_dtypes(include=['float64','int64'])
categorical_data = data_file.select_dtypes(exclude=['float64','int64'])
categorical_df = pd.DataFrame()
for column in categorical_data.columns:
    dummies = pd.get_dummies(categorical_data[column], prefix=column)
    categorical_df = pd.concat([categorical_df, dummies], axis=1)
#df = pd.concat([df['AMI_FLAG'], categorical_df], axis=1)
df = pd.concat([df, categorical_df], axis=1)

In [5]:
df.shape

(100000, 469)

In [6]:
sum(df['AMI_FLAG'] == 1)
df_AMI = df[df['AMI_FLAG'] == 1]
df_NOAMI = df[df['AMI_FLAG'] == 0]
df_NOAMI_sample = df_NOAMI.sample(2726)
print(df_AMI.shape)
print(df_NOAMI_sample.shape)

print()
df = pd.concat([df_AMI, df_NOAMI_sample], axis = 0)
# drop the null value 

print(df.shape)
df = df.dropna()
print(df.shape)

# define the target we would like to predict 
y = df['AMI_FLAG']

X = df.loc[:, df.columns != 'AMI_FLAG']

# sanity check on X, y dimensions 
print(X.shape)
print(y.shape)

# split training and testing data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=42)

(2726, 469)
(2726, 469)

(5452, 469)
(4049, 469)
(4049, 468)
(4049,)


In [6]:
# modeling 

x = tf.placeholder(dtype = tf.float32, shape = [None, 468])
y = tf.placeholder(dtype = tf.int32, shape = [None])


# tanh, elu, selu, softplus, and softsign

#logits = tf.contrib.layers.fully_connected(x, 128, tf.nn.relu)
#logits = tf.contrib.layers.fully_connected(x, 128, tf.nn.tanh)
#logits = tf.contrib.layers.fully_connected(x, 128, tf.nn.selu)


#logits = tf.contrib.layers.fully_connected(x, 128, tf.nn.softplus)
logits = tf.contrib.layers.fully_connected(x, 128, tf.nn.elu)


loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = logits))
train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
correct_pred = tf.argmax(logits, 1)
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

print("x: ", x)
print("logits: ", logits)
print("loss: ", loss)
print("predicted_labels: ", correct_pred)

x:  Tensor("Placeholder:0", shape=(?, 468), dtype=float32)
logits:  Tensor("fully_connected/Elu:0", shape=(?, 128), dtype=float32)
loss:  Tensor("Mean:0", shape=(), dtype=float32)
predicted_labels:  Tensor("ArgMax:0", shape=(?,), dtype=int64)


In [7]:
sess = tf.Session()

sess.run(tf.global_variables_initializer())

for i in range(201):
        #print('EPOCH', i)
        _, accuracy_val = sess.run([train_op, accuracy], feed_dict={x: X_train, y: y_train})
        if i % 50 == 0:
            print("Loss: ", loss)
        #print('DONE WITH EPOCH')

Loss:  Tensor("Mean:0", shape=(), dtype=float32)
Loss:  Tensor("Mean:0", shape=(), dtype=float32)
Loss:  Tensor("Mean:0", shape=(), dtype=float32)
Loss:  Tensor("Mean:0", shape=(), dtype=float32)
Loss:  Tensor("Mean:0", shape=(), dtype=float32)


In [8]:

predicted = sess.run([correct_pred], feed_dict={x: X_train})[0]

# Calculate correct matches 
match_count = sum([int(y == y_) for y, y_ in zip(y_train, predicted)])

# Calculate the accuracy
accuracy = match_count / len(X_train)

# Print the accuracy
print("Accuracy: {:.3f}".format(accuracy))


Accuracy: 0.485


In [9]:


predicted = sess.run([correct_pred], feed_dict={x: X_test})[0]

# Calculate correct matches 
match_count = sum([int(y == y_) for y, y_ in zip(y_test, predicted)])

# Calculate the accuracy
accuracy = match_count / len(y_test)

# Print the accuracy
print("Accuracy: {:.3f}".format(accuracy))

Accuracy: 0.515


In [10]:
# try some other sklearn model here

from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train, y_train)



pred_train = clf.predict(X_train)
pred_test = clf.predict(X_test)
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))


1.0
0.5834586466165413


In [11]:

from sklearn.svm import SVC

clf = SVC(kernel='rbf', gamma='auto')
clf.fit(X_train, y_train)

print('finish fitting data')


pred_train = clf.predict(X_train)

pred_test = clf.predict(X_test)
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))



finish fitting data
1.0
0.4857142857142857


In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification


clf = RandomForestClassifier(n_estimators=100, max_depth=2, criterion='entropy', random_state=0)
clf.fit(X_train, y_train)



print(clf.feature_importances_)


pred_train = clf.predict(X_train)
pred_test = clf.predict(X_test)
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))

[0.00061026 0.04471592 0.         0.04994351 0.05721379 0.00276385
 0.         0.002607   0.00482703 0.         0.         0.
 0.         0.00913151 0.         0.         0.         0.
 0.         0.         0.         0.00381836 0.00194719 0.
 0.         0.         0.00053705 0.         0.         0.
 0.00113192 0.         0.00111787 0.         0.00108165 0.00092652
 0.01792456 0.00106769 0.00630082 0.00263703 0.00125216 0.
 0.00026842 0.         0.00090228 0.         0.         0.
 0.00162783 0.00122255 0.00083549 0.00660314 0.         0.
 0.         0.0021903  0.         0.00441369 0.00264926 0.
 0.         0.         0.         0.00162199 0.00365885 0.00348728
 0.00199376 0.00125249 0.01379803 0.         0.         0.
 0.         0.         0.         0.         0.         0.00144695
 0.         0.00473379 0.         0.         0.         0.
 0.00085023 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0

In [11]:
important = clf.feature_importances_
extractedIdx = np.nonzero(important)

In [12]:
print(len(extractedIdx[0]))
extractedIdx

111


(array([  0,   1,   3,   4,   5,   7,   8,  13,  21,  22,  26,  30,  32,
         34,  35,  36,  37,  38,  39,  40,  42,  44,  48,  49,  50,  51,
         55,  57,  58,  63,  64,  65,  66,  67,  68,  77,  79,  84,  98,
        105, 111, 135, 136, 138, 139, 140, 142, 143, 145, 146, 149, 151,
        152, 154, 156, 158, 165, 174, 220, 223, 291, 292, 296, 298, 302,
        305, 307, 308, 309, 312, 314, 318, 319, 322, 323, 327, 332, 348,
        349, 356, 378, 379, 380, 381, 382, 383, 384, 387, 388, 389, 390,
        391, 393, 395, 396, 403, 405, 406, 410, 412, 414, 416, 417, 419,
        421, 424, 426, 437, 438, 440, 458]),)

In [21]:
# previously
print(len(extractedIdx[0]))
extractedIdx

107


(array([  0,   1,   2,   3,   4,   7,   8,  10,  12,  15,  25,  26,  35,
         36,  37,  39,  40,  46,  55,  56,  57,  58,  62,  63,  64,  65,
         66,  67,  68,  69,  79,  82,  93,  97,  98, 100, 105, 118, 134,
        139, 142, 143, 145, 146, 148, 149, 151, 152, 154, 158, 159, 161,
        170, 174, 175, 219, 221, 291, 296, 297, 298, 299, 300, 301, 304,
        305, 306, 307, 310, 312, 313, 319, 322, 323, 332, 334, 336, 346,
        378, 379, 380, 381, 382, 383, 387, 388, 390, 391, 393, 395, 396,
        397, 403, 409, 410, 412, 414, 416, 417, 419, 423, 424, 426, 434,
        436, 437, 458]),)

In [13]:
print(df.columns[0])
extracted_name = []
for idx in extractedIdx:
    extracted_name.append(df.columns[idx])

print(len(extracted_name[0]))
lst = list(extracted_name[0])
newDF = df[lst].copy()

ID
111


In [14]:
print(newDF.columns)

Index(['ID', 'AGE', 'ORIG_REAS_ENTITLE_CD', 'RECON_MA_RISK_SCORE_NBR',
       'RECON_RX_RISK_SCORE_NBR', 'CON_VISIT_04_Q02', 'CON_VISIT_04_Q03',
       'CON_VISIT_03_Q02', 'CON_VISIT_23_Q02', 'CON_VISIT_24_Q02',
       ...
       'Index_Health_ins_engage', 'Index_Health_ins_influence',
       'Population_density_centile_US', 'CDC_HBATEST_GAP', 'CDC_EYE_GAP',
       'BCS_GAP', 'SEX_CD_F', 'SEX_CD_M', 'ESRD_IND_Y',
       'Diab_Type_Diabetes Type I'],
      dtype='object', length=111)


In [ ]:
# CON_VISIT 11, 22; POT_VISIT_65_Q02 - Q03; 
# Population_density_centile_US; RENAL; Diab_Type_Diabetes Type I; 
# DIABETES
newDF['']

In [58]:
newDF.describe()

,ID,AGE,ORIG_REAS_ENTITLE_CD,RECON_MA_RISK_SCORE_NBR,CON_VISIT_04_Q03,CON_VISIT_21_Q03,CON_VISIT_03_Q02,CON_VISIT_03_Q04,CON_VISIT_30_Q02,CON_VISIT_30_Q04,...,Length_residence,Est_Net_worth,CDC,CDC_EYE_GAP,BCS_GAP,COL_GAP,STATIN_ELIG,SEX_CD_F,HOSPICE_IND_Y,Diab_Type_Diabetes Type I
count,4023.000000,4023.000000,4023.000000,4023.000000,4023.000000,4023.000000,4023.000000,4023.000000,4023.000000,4023.000000,...,4023.000000,4023.000000,4023.000000,4023.000000,4023.000000,4023.000000,4023.000000,4023.00000,4023.000000,4023.000000
mean,50217.926423,74.544370,0.279891,1.378850,0.534178,0.891872,0.276162,0.282376,0.121800,0.103157,...,16.138454,211311.210539,0.239125,0.103903,0.051454,0.160825,0.478747,0.52349,0.003231,0.037534
std,28973.357908,9.446673,0.450107,1.103054,1.715684,2.597763,1.623744,1.281706,1.220201,0.838558,...,17.629156,275881.935722,0.426602,0.305172,0.220950,0.367415,0.499610,0.49951,0.056761,0.190090
min,9.000000,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-2500.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,24980.000000,69.000000,0.000000,0.631500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.000000,2500.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
50%,50330.000000,74.000000,0.000000,1.034000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,13.000000,87500.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000,0.000000,0.000000
75%,75125.500000,81.000000,1.000000,1.759500,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,19.000000,312500.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.00000,0.000000,0.000000
max,99960.000000,95.000000,2.000000,8.295000,38.000000,34.000000,43.000000,21.000000,60.000000,27.000000,...,99.000000,1000000.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000


In [15]:
newDF.shape

(4049, 111)

In [16]:
df.shape

(4049, 469)

In [17]:
len(y)

4049

In [18]:


newDF['AMI_FLAG'] = pd.Series(y, index=newDF.index)
# use only the 107 features, retrain models
df = newDF
print(df.shape)
df = df.dropna()
print(df.shape)

# define the target we would like to predict 
y = df['AMI_FLAG']

X = df.loc[:, df.columns != 'AMI_FLAG']

# sanity check on X, y dimensions 
print(X.shape)
print(y.shape)

# split training and testing data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=42)

(4049, 112)
(4049, 112)
(4049, 111)
(4049,)


In [19]:
df.shape[1]

112

In [66]:
# modeling 

x = tf.placeholder(dtype = tf.float32, shape = [None, (df.shape[1]-1)])
y = tf.placeholder(dtype = tf.int32, shape = [None])


# tanh, elu, selu, softplus, and softsign

#logits = tf.contrib.layers.fully_connected(x, 50, tf.nn.elu)
#logits = tf.contrib.layers.fully_connected(x, 128, tf.nn.tanh)
#logits = tf.contrib.layers.fully_connected(x, 128, tf.nn.selu)


# this two model has better accuracy -------------

logits = tf.contrib.layers.fully_connected(x, 50, tf.nn.relu)
logits = tf.contrib.layers.fully_connected(x, 50, tf.nn.softplus)



loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = logits))
train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
correct_pred = tf.argmax(logits, 1)
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

print("x: ", x)
print("logits: ", logits)
print("loss: ", loss)
print("predicted_labels: ", correct_pred)

x:  Tensor("Placeholder_2:0", shape=(?, 109), dtype=float32)
logits:  Tensor("fully_connected_2/Softplus:0", shape=(?, 50), dtype=float32)
loss:  Tensor("Mean_2:0", shape=(), dtype=float32)
predicted_labels:  Tensor("ArgMax_1:0", shape=(?,), dtype=int64)


In [67]:
sess = tf.Session()

sess.run(tf.global_variables_initializer())

for i in range(201):
        #print('EPOCH', i)
        _, accuracy_val = sess.run([train_op, accuracy], feed_dict={x: X_train, y: y_train})
        if i % 50 == 0:
            print("Loss: ", loss)
        #print('DONE WITH EPOCH')

Loss:  Tensor("Mean_2:0", shape=(), dtype=float32)
Loss:  Tensor("Mean_2:0", shape=(), dtype=float32)
Loss:  Tensor("Mean_2:0", shape=(), dtype=float32)
Loss:  Tensor("Mean_2:0", shape=(), dtype=float32)
Loss:  Tensor("Mean_2:0", shape=(), dtype=float32)


In [68]:

predicted = sess.run([correct_pred], feed_dict={x: X_train})[0]

# Calculate correct matches 
match_count = sum([int(y == y_) for y, y_ in zip(y_train, predicted)])

# Calculate the accuracy
accuracy = match_count / len(X_train)

# Print the accuracy
print("Accuracy: {:.3f}".format(accuracy))


Accuracy: 0.509


In [69]:


predicted = sess.run([correct_pred], feed_dict={x: X_test})[0]

# Calculate correct matches 
match_count = sum([int(y == y_) for y, y_ in zip(y_test, predicted)])

# Calculate the accuracy
accuracy = match_count / len(y_test)

# Print the accuracy
print("Accuracy: {:.3f}".format(accuracy))

Accuracy: 0.501


In [22]:



clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train, y_train)



pred_train = clf.predict(X_train)
pred_test = clf.predict(X_test)
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))


1.0
0.5706806282722513


In [219]:
# Here: testing decision tree with grid search 


# Number of features to consider at every split
max_features = ['auto', 'sqrt', 'log2']
# Maximum number of levels in tree
max_depth = [10, 20, 30]
max_depth.append(None)
# Minimum number of samples required to split a node

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# crterion selecton
criterion = ['gini', 'entropy']

# Create the random grid
random_grid = {'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_leaf': min_samples_leaf,
               'criterion': criterion}

print(random_grid)


{'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 20, 30, None], 'min_samples_leaf': [1, 2, 4], 'criterion': ['gini', 'entropy']}


In [220]:
clf = RandomForestClassifier()

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = clf, 
                               param_distributions = random_grid, 
                               n_iter = 72, cv = 3, verbose=2, 
                               random_state=42, n_jobs = -1)

In [221]:
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 72 candidates, totalling 216 fits
[CV] min_samples_leaf=1, max_features=auto, max_depth=10, criterion=gini 
[CV] min_samples_leaf=1, max_features=auto, max_depth=10, criterion=gini 
[CV] min_samples_leaf=1, max_features=auto, max_depth=10, criterion=gini 
[CV]  min_samples_leaf=1, max_features=auto, max_depth=10, criterion=gini, total=   0.1s
[CV] min_samples_leaf=2, max_features=auto, max_depth=10, criterion=gini 
[CV]  min_samples_leaf=1, max_features=auto, max_depth=10, criterion=gini, total=   0.1s
[CV] min_samples_leaf=2, max_features=auto, max_depth=10, criterion=gini 
[CV] min_samples_leaf=2, max_features=auto, max_depth=10, criterion=gini 
[CV]  min_samples_leaf=1, max_features=auto, max_depth=10, criterion=gini, total=   0.1s
[CV] min_samples_leaf=4, max_features=auto, max_depth=10, criterion=gini 
[CV]  min_samples_leaf=2, max_features=auto, max_depth=10, criterion=gini, total=   0.1s
[CV] min_samples_leaf=4, max_features=auto, max_depth=10, criter

[CV] min_samples_leaf=2, max_features=log2, max_depth=20, criterion=gini 
[CV]  min_samples_leaf=2, max_features=log2, max_depth=20, criterion=gini, total=   0.1s
[CV] min_samples_leaf=4, max_features=log2, max_depth=20, criterion=gini 
[CV]  min_samples_leaf=1, max_features=log2, max_depth=20, criterion=gini, total=   0.1s
[CV]  min_samples_leaf=4, max_features=log2, max_depth=20, criterion=gini, total=   0.1s
[CV]  min_samples_leaf=1, max_features=log2, max_depth=20, criterion=gini, total=   0.1s
[CV] min_samples_leaf=4, max_features=log2, max_depth=20, criterion=gini 
[CV] min_samples_leaf=4, max_features=auto, max_depth=30, criterion=gini 
[CV] min_samples_leaf=4, max_features=auto, max_depth=30, criterion=gini 
[CV]  min_samples_leaf=1, max_features=auto, max_depth=30, criterion=gini, total=   0.1s
[CV]  min_samples_leaf=1, max_features=auto, max_depth=30, criterion=gini, total=   0.1s
[CV] min_samples_leaf=1, max_features=auto, max_depth=30, criterion=gini 
[CV] min_samples_leaf=

[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.7s


[CV]  min_samples_leaf=2, max_features=auto, max_depth=30, criterion=gini, total=   0.1s
[CV]  min_samples_leaf=4, max_features=log2, max_depth=20, criterion=gini, total=   0.1s
[CV] min_samples_leaf=2, max_features=auto, max_depth=30, criterion=gini 
[CV] min_samples_leaf=1, max_features=sqrt, max_depth=30, criterion=gini 
[CV]  min_samples_leaf=4, max_features=log2, max_depth=20, criterion=gini, total=   0.1s
[CV] min_samples_leaf=2, max_features=sqrt, max_depth=30, criterion=gini 
[CV]  min_samples_leaf=4, max_features=auto, max_depth=30, criterion=gini, total=   0.1s
[CV] min_samples_leaf=4, max_features=auto, max_depth=30, criterion=gini 
[CV]  min_samples_leaf=4, max_features=auto, max_depth=30, criterion=gini, total=   0.1s
[CV] min_samples_leaf=2, max_features=sqrt, max_depth=30, criterion=gini 
[CV] min_samples_leaf=1, max_features=sqrt, max_depth=30, criterion=gini 
[CV]  min_samples_leaf=1, max_features=auto, max_depth=30, criterion=gini, total=   0.1s
[CV]  min_samples_leaf

[CV]  min_samples_leaf=4, max_features=log2, max_depth=None, criterion=gini, total=   0.1s
[CV] min_samples_leaf=4, max_features=auto, max_depth=10, criterion=entropy 
[CV] min_samples_leaf=4, max_features=log2, max_depth=None, criterion=gini 
[CV]  min_samples_leaf=2, max_features=log2, max_depth=None, criterion=gini, total=   0.1s
[CV]  min_samples_leaf=1, max_features=auto, max_depth=10, criterion=entropy, total=   0.1s
[CV] min_samples_leaf=1, max_features=auto, max_depth=10, criterion=entropy 
[CV] min_samples_leaf=1, max_features=sqrt, max_depth=10, criterion=entropy 
[CV]  min_samples_leaf=2, max_features=auto, max_depth=10, criterion=entropy, total=   0.1s
[CV] min_samples_leaf=2, max_features=auto, max_depth=10, criterion=entropy 
[CV]  min_samples_leaf=4, max_features=auto, max_depth=10, criterion=entropy, total=   0.1s
[CV] min_samples_leaf=4, max_features=auto, max_depth=10, criterion=entropy 
[CV]  min_samples_leaf=4, max_features=log2, max_depth=None, criterion=gini, tota

[CV] min_samples_leaf=1, max_features=auto, max_depth=30, criterion=entropy 
[CV] min_samples_leaf=1, max_features=log2, max_depth=20, criterion=entropy 
[CV]  min_samples_leaf=2, max_features=log2, max_depth=20, criterion=entropy, total=   0.1s
[CV] min_samples_leaf=2, max_features=log2, max_depth=20, criterion=entropy 
[CV]  min_samples_leaf=2, max_features=log2, max_depth=20, criterion=entropy, total=   0.1s
[CV] min_samples_leaf=4, max_features=log2, max_depth=20, criterion=entropy 
[CV]  min_samples_leaf=4, max_features=sqrt, max_depth=20, criterion=entropy, total=   0.1s
[CV]  min_samples_leaf=4, max_features=log2, max_depth=20, criterion=entropy, total=   0.1s
[CV] min_samples_leaf=4, max_features=log2, max_depth=20, criterion=entropy 
[CV] min_samples_leaf=2, max_features=auto, max_depth=30, criterion=entropy 
[CV]  min_samples_leaf=1, max_features=auto, max_depth=30, criterion=entropy, total=   0.1s
[CV]  min_samples_leaf=1, max_features=log2, max_depth=20, criterion=entropy, 

[CV] min_samples_leaf=2, max_features=sqrt, max_depth=None, criterion=entropy 
[CV]  min_samples_leaf=4, max_features=sqrt, max_depth=None, criterion=entropy, total=   0.1s
[CV] min_samples_leaf=4, max_features=sqrt, max_depth=None, criterion=entropy 
[CV]  min_samples_leaf=4, max_features=sqrt, max_depth=None, criterion=entropy, total=   0.1s
[CV] min_samples_leaf=1, max_features=log2, max_depth=None, criterion=entropy 
[CV]  min_samples_leaf=1, max_features=log2, max_depth=None, criterion=entropy, total=   0.1s
[CV] min_samples_leaf=1, max_features=log2, max_depth=None, criterion=entropy 
[CV]  min_samples_leaf=2, max_features=log2, max_depth=None, criterion=entropy, total=   0.1s
[CV] min_samples_leaf=2, max_features=log2, max_depth=None, criterion=entropy 
[CV]  min_samples_leaf=2, max_features=log2, max_depth=None, criterion=entropy, total=   0.1s
[CV]  min_samples_leaf=1, max_features=sqrt, max_depth=None, criterion=entropy, total=   0.1s
[CV] min_samples_leaf=4, max_features=log

[Parallel(n_jobs=-1)]: Done 216 out of 216 | elapsed:    3.2s finished


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid=True, n_iter=72, n_jobs=-1,
          param_distributions={'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 20, 30, None], 'min_samples_leaf': [1, 2, 4], 'criterion': ['gini', 'entropy']},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [222]:
rf_random.best_params_

{'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 10,
 'criterion': 'gini'}

In [23]:
clf = DecisionTreeClassifier(min_samples_leaf=4, max_features='sqrt', 
                             max_depth=10, random_state=42, criterion='gini')
half_clf = DecisionTreeClassifier(min_samples_leaf=4, max_features='sqrt', 
                             max_depth=10, random_state=42, criterion='gini')

In [242]:
half_clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
            max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

In [243]:
clf.fit(whole_X_train, whole_y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
            max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

In [244]:
# unbalanced classifier, predict on unbalanced data set

pred_train = clf.predict(whole_X_train)
pred_test = clf.predict(whole_X_test)
print(accuracy_score(whole_y_train, pred_train))
print(accuracy_score(whole_y_test, pred_test))

0.9906168895987223
0.9891374837872893


In [245]:

# balanced classifier, predict on balanced data set
half_pred_train = half_clf.predict(X_train)
half_pred_test = half_clf.predict(X_test)
print(accuracy_score(y_train, half_pred_train))
print(accuracy_score(y_test, half_pred_test))



# unbalanced classifier, predict on unbalanced data set
whole_pred_train = clf.predict(whole_X_train)
whole_pred_test = clf.predict(whole_X_test)

print(accuracy_score(whole_y_train, whole_pred_train))
print(accuracy_score(whole_y_test, whole_pred_test))

0.7531089978054133
0.6095025983667409
0.9906168895987223
0.9891374837872893


In [246]:
# balanced classifier, predict on unbalanced dataset

half_pred_train = half_clf.predict(whole_X_train)
half_pred_test = half_clf.predict(whole_X_test)

print(accuracy_score(whole_y_train, half_pred_train))
print(accuracy_score(whole_y_test, half_pred_test))

0.8462367738071471
0.8471952010376135


In [234]:
# unbalanced classifier, predict on balanced dataset

pred_train = clf.predict(X_train)
pred_test = clf.predict(X_test)

print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))

0.5307242136064374
0.49443207126948774


In [71]:
clf = SVC(kernel='rbf', gamma='auto')
clf.fit(X_train, y_train)

print('finish fitting data')


pred_train = clf.predict(X_train)

pred_test = clf.predict(X_test)
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))

finish fitting data
1.0
0.48268072289156627


In [72]:

clf = RandomForestClassifier(n_estimators=100, max_depth=2, criterion='entropy', random_state=0)
clf.fit(X_train, y_train)



print(clf.feature_importances_)


pred_train = clf.predict(X_train)
pred_test = clf.predict(X_test)
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))

[0.00238801 0.0543412  0.         0.09734659 0.         0.00043926
 0.         0.00333447 0.         0.         0.0002895  0.00098941
 0.         0.00385695 0.00616293 0.01821779 0.00362206 0.00901828
 0.00144248 0.00059018 0.00079547 0.02541101 0.00031664 0.00029459
 0.00248738 0.00193534 0.00514849 0.         0.00103364 0.
 0.00071149 0.         0.00050426 0.         0.00083876 0.00170695
 0.0077444  0.00071195 0.00908005 0.00132187 0.00832242 0.02381241
 0.0071825  0.         0.         0.0192523  0.00179057 0.0009103
 0.         0.         0.01058999 0.00081268 0.         0.
 0.00037833 0.         0.         0.00060358 0.00233617 0.00061518
 0.         0.0047635  0.00899555 0.02028387 0.         0.00156675
 0.0014624  0.00067006 0.00081965 0.01857982 0.00284107 0.
 0.00062783 0.         0.00081742 0.02008973 0.00054156 0.
 0.00025157 0.         0.         0.         0.         0.16118625
 0.06384068 0.09157049 0.02218334 0.05616918 0.04684235 0.
 0.04143103 0.00297697 0.02590653 0.

In [78]:
y_data = df['AMI_FLAG']

In [80]:
len(y_data)

4023

In [83]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(clf, X, y_data, cv = 5)

In [84]:
scores

array([0.43796526, 0.66583851, 0.6641791 , 0.66169154, 0.39552239])

In [29]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [200, 300, 400, 500, 600]
# Number of features to consider at every split
max_features = ['auto', 'sqrt', 'log2']
# Maximum number of levels in tree
max_depth = [10, 20, 30]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]



# crterion selecton
criterion = ['gini', 'entropy']

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'criterion': criterion}

print(random_grid)

{'n_estimators': [200, 300, 400, 500, 600], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 20, 30, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'criterion': ['gini', 'entropy']}


In [30]:
clf = RandomForestClassifier()

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = clf, 
                               param_distributions = random_grid, 
                               n_iter = 100, cv = 3, verbose=2, 
                               random_state=42, n_jobs = -1)

In [31]:
rf_random.cv_results_

AttributeError: 'RandomizedSearchCV' object has no attribute 'cv_results_'

In [107]:
rf_random.get_params

<bound method BaseEstimator.get_params of RandomizedSearchCV(cv=3, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid=True, n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [200, 300, 400, 500, 600], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 20, 30, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'criterion': ['gini', 'entropy']},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)>

In [32]:
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] n_estimators=500, min_samples_split=2, min_samples_leaf=2, max_features=auto, max_depth=None, criterion=gini 
[CV] n_estimators=500, min_samples_split=2, min_samples_leaf=2, max_features=auto, max_depth=None, criterion=gini 
[CV] n_estimators=500, min_samples_split=2, min_samples_leaf=2, max_features=auto, max_depth=None, criterion=gini 
[CV] n_estimators=500, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=10, criterion=entropy 
[CV] n_estimators=500, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=10, criterion=entropy 
[CV] n_estimators=500, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=10, criterion=entropy 
[CV] n_estimators=300, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=20, criterion=entropy 
[CV] n_estimators=300, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=20, criterion=entropy 
[CV]  n_es

[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   11.7s


[CV]  n_estimators=500, min_samples_split=2, min_samples_leaf=1, max_features=sqrt, max_depth=None, criterion=entropy, total=   5.1s
[CV]  n_estimators=500, min_samples_split=2, min_samples_leaf=1, max_features=sqrt, max_depth=None, criterion=entropy, total=   5.1s
[CV] n_estimators=300, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=10, criterion=entropy 
[CV] n_estimators=300, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=10, criterion=entropy 
[CV]  n_estimators=500, min_samples_split=5, min_samples_leaf=2, max_features=log2, max_depth=30, criterion=entropy, total=   3.8s
[CV] n_estimators=300, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=10, criterion=entropy 
[CV]  n_estimators=500, min_samples_split=5, min_samples_leaf=2, max_features=log2, max_depth=30, criterion=entropy, total=   3.8s
[CV] n_estimators=600, min_samples_split=2, min_samples_leaf=1, max_features=log2, max_depth=10, criterion=entropy 
[CV] 

[CV] n_estimators=200, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=None, criterion=gini 
[CV]  n_estimators=500, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=30, criterion=entropy, total=   4.7s
[CV] n_estimators=200, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=None, criterion=gini 
[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=10, criterion=gini, total=   1.4s
[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=10, criterion=gini, total=   1.4s
[CV] n_estimators=200, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=None, criterion=gini 
[CV] n_estimators=500, min_samples_split=10, min_samples_leaf=2, max_features=log2, max_depth=30, criterion=gini 
[CV]  n_estimators=300, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=10, criterion=gini, total=   2.3s
[CV] n_estimators=500, 

[CV]  n_estimators=600, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=None, criterion=entropy, total=   4.9s
[CV] n_estimators=400, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=30, criterion=gini 
[CV]  n_estimators=300, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=10, criterion=gini, total=   2.2s
[CV] n_estimators=400, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=30, criterion=gini 
[CV]  n_estimators=300, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=10, criterion=gini, total=   2.2s
[CV] n_estimators=600, min_samples_split=5, min_samples_leaf=4, max_features=auto, max_depth=20, criterion=gini 
[CV]  n_estimators=300, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=10, criterion=gini, total=   2.1s
[CV] n_estimators=600, min_samples_split=5, min_samples_leaf=4, max_features=auto, max_depth=20, criterion=gini 
[CV]  n_estimators=600, min_sam

...

[CV]  n_estimators=300, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=10, criterion=entropy, total=   1.8s
[CV] n_estimators=500, min_samples_split=5, min_samples_leaf=2, max_features=log2, max_depth=10, criterion=gini 
[CV]  n_estimators=300, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=10, criterion=entropy, total=   1.8s
[CV] n_estimators=500, min_samples_split=5, min_samples_leaf=2, max_features=log2, max_depth=10, criterion=gini 
[CV]  n_estimators=400, min_samples_split=2, min_samples_leaf=2, max_features=log2, max_depth=30, criterion=gini, total=   2.2s
[CV] n_estimators=500, min_samples_split=5, min_samples_leaf=2, max_features=log2, max_depth=10, criterion=gini 
[CV]  n_estimators=400, min_samples_split=2, min_samples_leaf=2, max_features=log2, max_depth=30, criterion=gini, total=   2.2s
[CV] n_estimators=500, min_samples_split=5, min_samples_leaf=4, max_features=auto, max_depth=10, criterion=entropy 
[CV]  n_estimators=600, m

[CV] n_estimators=300, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=30, criterion=gini 
[CV]  n_estimators=400, min_samples_split=2, min_samples_leaf=2, max_features=log2, max_depth=10, criterion=gini, total=   1.9s
[CV] n_estimators=300, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=30, criterion=gini 
[CV]  n_estimators=300, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=20, criterion=entropy, total=   2.3s
[CV] n_estimators=300, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=30, criterion=gini 
[CV]  n_estimators=300, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=20, criterion=entropy, total=   2.3s
[CV] n_estimators=500, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=20, criterion=entropy 
[CV]  n_estimators=600, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=10, criterion=gini, total=   3.5s
[CV] n_estimators=500, 

[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=30, criterion=gini, total=   1.1s
[CV] n_estimators=400, min_samples_split=2, min_samples_leaf=4, max_features=sqrt, max_depth=30, criterion=entropy 
[CV]  n_estimators=600, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=None, criterion=gini, total=   3.3s
[CV] n_estimators=400, min_samples_split=2, min_samples_leaf=4, max_features=sqrt, max_depth=30, criterion=entropy 
[CV]  n_estimators=600, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=None, criterion=gini, total=   3.5s
[CV] n_estimators=600, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=None, criterion=gini 
[CV]  n_estimators=600, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=None, criterion=gini, total=   3.3s
[CV] n_estimators=600, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=None, criterion=gini 
[CV]  n_estimator

[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.0min finished


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid=True, n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [200, 300, 400, 500, 600], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 20, 30, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'criterion': ['gini', 'entropy']},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [33]:
rf_random.best_params_

{'n_estimators': 600,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': None,
 'criterion': 'entropy'}

In [34]:
rf_random.best_params_['n_estimators']

600

In [72]:
rf_random.best_params_

{'n_estimators': 600,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': None,
 'criterion': 'entropy'}

In [95]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))

    return accuracy

In [36]:
clf = RandomForestClassifier(n_estimators = rf_random.best_params_['n_estimators'], 
                                    min_samples_leaf= rf_random.best_params_['min_samples_leaf'], 
                                    min_samples_split= rf_random.best_params_['min_samples_split'], 
                                    max_depth = rf_random.best_params_['max_depth'], 
                                    max_features = rf_random.best_params_['max_features'],
                                    criterion = rf_random.best_params_['criterion'],
                                    bootstrap = True,
                                    random_state = 42)

In [37]:
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=5,
            min_weight_fraction_leaf=0.0, n_estimators=600, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [38]:
pred_train = clf.predict(X_train)
pred_test = clf.predict(X_test)
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))

0.9992641648270787
0.6318147871545929


Compare two training sets result:

In [39]:

# only select the numeric data
df = data_file.select_dtypes(include=['float64','int64'])
categorical_data = data_file.select_dtypes(exclude=['float64','int64'])
categorical_df = pd.DataFrame()
for column in categorical_data.columns:
    dummies = pd.get_dummies(categorical_data[column], prefix=column)
    categorical_df = pd.concat([categorical_df, dummies], axis=1)
#df = pd.concat([df['AMI_FLAG'], categorical_df], axis=1)
df = pd.concat([df, categorical_df], axis=1)

sum(df['AMI_FLAG'] == 1)
df_AMI = df[df['AMI_FLAG'] == 1]
df_NOAMI = df[df['AMI_FLAG'] == 0]
df_NOAMI_sample = df_NOAMI.sample(2726)
print(df_AMI.shape)
print(df_NOAMI_sample.shape)

print()
df = pd.concat([df_AMI, df_NOAMI_sample], axis = 0)
# drop the null value 

print(df.shape)
df = df.dropna()
print(df.shape)

# define the target we would like to predict 
y = df['AMI_FLAG']

X = df.loc[:, df.columns != 'AMI_FLAG']

# sanity check on X, y dimensions 
print(X.shape)
print(y.shape)

# split training and testing data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=42)

(2726, 469)
(2726, 469)

(5452, 469)
(4068, 469)
(4068, 468)
(4068,)


In [40]:
print(len(extracted_name[0]))
lst = list(extracted_name[0])
newDF = df[lst].copy()

111


In [41]:
newDF['AMI_FLAG'] = pd.Series(y, index=newDF.index)
# use only the 107 features, retrain models
df = newDF
print(df.shape)
df = df.dropna()
print(df.shape)

# define the target we would like to predict 
y = df['AMI_FLAG']

X = df.loc[:, df.columns != 'AMI_FLAG']

# sanity check on X, y dimensions 
print(X.shape)
print(y.shape)

# split training and testing data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=42)

(4068, 112)
(4068, 112)
(4068, 111)
(4068,)


In [60]:
half_clf = RandomForestClassifier(n_estimators = rf_random.best_params_['n_estimators'], 
                                    min_samples_leaf= rf_random.best_params_['min_samples_leaf'], 
                                    min_samples_split= rf_random.best_params_['min_samples_split'], 
                                    max_depth = rf_random.best_params_['max_depth'], 
                                    max_features = rf_random.best_params_['max_features'],
                                    criterion = rf_random.best_params_['criterion'],
                                    bootstrap = True,
                                    random_state = 42)

In [61]:
half_clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=5,
            min_weight_fraction_leaf=0.0, n_estimators=600, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [62]:
X_train.shape

(2725, 111)

In [63]:
pred_train = half_clf.predict(X_train)
pred_test = half_clf.predict(X_test)
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))

0.9992660550458715
0.6582278481012658


In [87]:
# test on the whole dataset

origDF = data_file.select_dtypes(include=['float64','int64'])
categorical_data = data_file.select_dtypes(exclude=['float64','int64'])
categorical_df = pd.DataFrame()
for column in categorical_data.columns:
    dummies = pd.get_dummies(categorical_data[column], prefix=column)
    categorical_df = pd.concat([categorical_df, dummies], axis=1)
#df = pd.concat([df['AMI_FLAG'], categorical_df], axis=1)
origDF = pd.concat([origDF, categorical_df], axis=1)

In [90]:
origDF = origDF.dropna()

origDF.shape

(74762, 111)

In [88]:
# define the target we would like to predict 
orig_y = origDF['AMI_FLAG']
orig_X = origDF.loc[:, df.columns != 'AMI_FLAG']

print(origDF.columns[0])
print(len(extracted_name[0]))
lst = list(extracted_name[0])
origDF = origDF[lst].copy()

ID
111


In [91]:
origDF.shape

(74762, 111)

In [68]:
orig_y.shape

(74762,)

In [51]:
orig_X.shape

(74762, 111)

In [69]:
whole_X_train, whole_X_test, whole_y_train, whole_y_test = train_test_split(orig_X, orig_y, test_size = 0.33, random_state=42)

In [70]:
clf = RandomForestClassifier(n_estimators = rf_random.best_params_['n_estimators'], 
                                    min_samples_leaf= rf_random.best_params_['min_samples_leaf'], 
                                    min_samples_split= rf_random.best_params_['min_samples_split'], 
                                    max_depth = rf_random.best_params_['max_depth'], 
                                    max_features = rf_random.best_params_['max_features'],
                                    criterion = rf_random.best_params_['criterion'],
                                    bootstrap = True,
                                    random_state = 42)

clf.get_params

<bound method BaseEstimator.get_params of RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=5,
            min_weight_fraction_leaf=0.0, n_estimators=600, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)>

In [71]:
clf.fit(whole_X_train, whole_y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=5,
            min_weight_fraction_leaf=0.0, n_estimators=600, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [73]:
pred_train = clf.predict(whole_X_train)
pred_test = clf.predict(whole_X_test)
print(accuracy_score(whole_y_train, pred_train))
print(accuracy_score(whole_y_test, pred_test))

1.0
1.0


In [74]:
half_pred_train = half_clf.predict(X_train)
half_pred_test = half_clf.predict(X_test)

whole_pred_train = clf.predict(whole_X_train)
whole_pred_test = clf.predict(whole_X_test)

print(accuracy_score(y_train, half_pred_train))
print(accuracy_score(y_test, half_pred_test))

print(accuracy_score(whole_y_train, whole_pred_train))
print(accuracy_score(whole_y_test, whole_pred_test))

0.9992660550458715
0.6582278481012658
1.0
1.0


In [75]:
whole_X_train.shape

(50090, 111)

In [77]:
half_pred_train = half_clf.predict(whole_X_train)
half_pred_test = half_clf.predict(whole_X_test)

print(accuracy_score(whole_y_train, half_pred_train))
print(accuracy_score(whole_y_test, half_pred_test))

0.9269315232581353
0.9236381322957199


In [211]:
half_pred_train = half_clf.predict(whole_X_train)
half_pred_test = half_clf.predict(whole_X_test)

print(accuracy_score(whole_y_train, half_pred_train))
print(accuracy_score(whole_y_test, half_pred_test))

0.9001597125174685
0.8982247081712063


In [78]:
pred_train = clf.predict(X_train)
pred_test = clf.predict(X_test)

print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))

0.531743119266055
0.526433358153388
